In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
import torch
from transformers eimport BertForSequenceClassification, Trainer, TrainingArguments

In [ ]:
#Step 1: Data Preprocessing

In [ ]:
data = pd.read_excel('WCMLDataset.xlsx')
example_data = data

In [ ]:
text_fields = [
    'Incident Description',
    'Activity Engaged in During Accident',
    'General HS Comments',
    'Injury Description'
]
example_data[text_fields] = example_data[text_fields].fillna('')

In [ ]:
# Combine text fields into a single input column
example_data['Combined_Text'] = (
    example_data['Incident Description'] + ' ' +
    example_data['Activity Engaged in During Accident'] + ' ' +
    example_data['General HS Comments'] + ' ' +
    example_data['Injury Description']
).str.lower().str.strip()

In [ ]:
# Encode target labels for all outputs
targets = ['Event of Injury Desc', 'Source of Injury Desc', 
           'Event of Incident Desc', 'Source of Incident Desc',
           'EDI Cause Desc']
label_encoders = {}

In [ ]:
for target in targets:
    le = LabelEncoder()
    example_data[target + '_Encoded'] = le.fit_transform(example_data[target])
    label_encoders[target] = le  # Save encoders for inverse transformation later

In [ ]:
#Step 2: Prepare Data for BERT

In [ ]:
# Initialize the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Split data for one target (e.g., "Source of Injury Desc")
X_train, X_test, y_train, y_test = train_test_split(
    example_data['Combined_Text'], 
    example_data['Source of Injury Desc_Encoded'], 
    test_size=0.2, 
    random_state=42
)

# Tokenize the text
train_encodings = tokenizer(list(X_train), truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(list(X_test), truncation=True, padding=True, max_length=512)

In [ ]:
# Convert tokenized data into a dataset format
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

# Prepare datasets for BERT
train_dataset = Dataset(train_encodings, list(y_train))
test_dataset = Dataset(test_encodings, list(y_test))

In [ ]:
#Step 3: Fine-Tuning BERT

In [ ]:
# Load pre-trained BERT model
num_labels = len(label_encoders['Source of Injury Desc'].classes_)
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_labels)

In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',          # Directory to save model checkpoints
    num_train_epochs=4,              # Number of training epochs
    per_device_train_batch_size=16,  # Batch size for training
    per_device_eval_batch_size=64,   # Batch size for evaluation
    warmup_steps=500,                # Number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # Strength of weight decay
    logging_dir='./logs',            # Directory for logs
    logging_steps=10,                # Log every 10 steps
    evaluation_strategy="epoch",     # Evaluate at the end of each epoch
    save_strategy="epoch",           # Save model at the end of each epoch
    load_best_model_at_end=True,     # Load the best model at the end of training
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [ ]:
trainer.train()

In [ ]:
# Save the fine-tuned model
model.save_pretrained('./fine_tuned_source_of_injury')

In [ ]:
#Step 4: Inference and Predictions

In [ ]:
# Load the fine-tuned model
model = BertForSequenceClassification.from_pretrained('./fine_tuned_source_of_injury')

# Tokenize new input text for prediction
new_text = ["He felt pain in his hand and wrists develop as the cooler door's automatic mechanism was broken."]
new_encodings = tokenizer(new_text, truncation=True, padding=True, max_length=512, return_tensors='pt')

In [ ]:
# Predict
model.eval()
outputs = model(**new_encodings)
predicted_class = torch.argmax(outputs.logits, dim=1).item()

In [ ]:
# Decode the predicted class
decoded_class = label_encoders['Source of Injury Desc'].inverse_transform([predicted_class])
print(f"Predicted Source of Injury: {decoded_class[0]}")